In [121]:
import pandas as pd
import re
import unicodedata
import datetime

In [122]:
df = pd.read_csv("../data/GoodReads_100k_books.csv")

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   author        100000 non-null  object 
 1   bookformat    96772 non-null   object 
 2   desc          93228 non-null   object 
 3   genre         89533 non-null   object 
 4   img           96955 non-null   object 
 5   isbn          85518 non-null   object 
 6   isbn13        88565 non-null   object 
 7   link          100000 non-null  object 
 8   pages         100000 non-null  int64  
 9   rating        100000 non-null  float64
 10  reviews       100000 non-null  int64  
 11  title         99999 non-null   object 
 12  totalratings  100000 non-null  int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 9.9+ MB


In [124]:
cols_to_keep = [
    "title",
    "desc",
    "bookformat",
    "pages",
    "rating",
    "totalratings",
    "reviews",
    "isbn",
    "isbn13",
    "img",
    "link",
    "author",
    "genre",
]
df = df[cols_to_keep]

In [125]:
df = df.dropna(subset=["title", "isbn", "desc", "author", "genre"])

In [126]:
df = df.drop_duplicates(subset=["isbn", "title"])

In [127]:
df["pages"] = df["pages"].astype(int)
df["rating"] = df["rating"].astype(float)
df["totalratings"] = df["totalratings"].astype(int)
df["reviews"] = df["reviews"].astype(int)

In [128]:
df = df[(df["rating"] >= 0) & (df["rating"] <= 5)]

In [129]:
df["title"] = df["title"].str.strip()
df["author"] = df["author"].str.strip()
df["genre"] = df["genre"].str.strip()

In [130]:
df = df.reset_index(drop=True)

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74141 entries, 0 to 74140
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         74141 non-null  object 
 1   desc          74141 non-null  object 
 2   bookformat    73415 non-null  object 
 3   pages         74141 non-null  int64  
 4   rating        74141 non-null  float64
 5   totalratings  74141 non-null  int64  
 6   reviews       74141 non-null  int64  
 7   isbn          74141 non-null  object 
 8   isbn13        73253 non-null  object 
 9   img           73719 non-null  object 
 10  link          74141 non-null  object 
 11  author        74141 non-null  object 
 12  genre         74141 non-null  object 
dtypes: float64(1), int64(3), object(9)
memory usage: 7.4+ MB


In [132]:
# ensure no nulls before splitting
df["author"] = df["author"].fillna("").astype(str)
df["genre"] = df["genre"].fillna("").astype(str)

# split by comma and strip extra spaces
df["author_list"] = df["author"].apply(lambda x: [a.strip() for a in x.split(",") if a.strip()])
df["genre_list"] = df["genre"].apply(lambda x: [g.strip() for g in x.split(",") if g.strip()])

In [133]:
df.head()

,title,desc,bookformat,pages,rating,totalratings,reviews,isbn,isbn13,img,link,author,genre,author_list,genre_list
0,Between Two Fires: American Indians in the Civ...,Reveals that several hundred thousand Indians ...,Hardcover,0,3.52,33,5,002914180X,9.78E+12,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/1001053.Betwee...,Laurence M. Hauptman,"History,Military History,Civil War,American Hi...",[Laurence M. Hauptman],"[History, Military History, Civil War, America..."
1,Fashion Sourcebook 1920s,Fashion Sourcebook - 1920s is the first book i...,Paperback,576,4.51,41,6,1906863482,9.78E+12,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/10010552-fashi...,"Charlotte Fiell,Emmanuelle Dirix","Couture,Fashion,Historical,Art,Nonfiction","[Charlotte Fiell, Emmanuelle Dirix]","[Couture, Fashion, Historical, Art, Nonfiction]"
2,Hungary 56,The seminal history and analysis of the Hungar...,Paperback,124,4.15,26,2,948984147,9.78E+12,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/1001077.Hungar...,Andy Anderson,"Politics,History",[Andy Anderson],"[Politics, History]"
3,All-American Anarchist: Joseph A. Labadie and ...,"""All-American Anarchist"" chronicles the life a...",Hardcover,324,3.83,6,1,814327079,9.78E+12,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/1001079.All_Am...,Carlotta R. Anderson,"Labor,History",[Carlotta R. Anderson],"[Labor, History]"
4,The Human Equation: Building Profits by Puttin...,Why is common sense so uncommon when it comes ...,Hardcover,368,3.73,119,7,875848419,9.78E+12,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/1001090.The_Hu...,Jeffrey Pfeffer,"Business,Leadership,Romance,Historical Romance...",[Jeffrey Pfeffer],"[Business, Leadership, Romance, Historical Rom..."


In [134]:
ascii_clean_pattern = re.compile(r"^[A-Za-z0-9\s.,!?;:'\"()\-_/&]+$")


def normalize_and_check_ascii(text):
    if not text or not isinstance(text, str):
        return False
    # normalize text to remove weird unicode
    text = unicodedata.normalize("NFKD", text)
    # replace non-breaking spaces or invisible chars
    text = re.sub(r"[\u200b\u200c\u200d\uFEFF]", "", text)
    text = text.strip()
    return bool(ascii_clean_pattern.match(text))

In [135]:
def row_has_bad_text(row):
    return any(not normalize_and_check_ascii(a) for a in row["author_list"]) or any(
        not normalize_and_check_ascii(g) for g in row["genre_list"]
    )


df["has_bad_text"] = df.apply(row_has_bad_text, axis=1)

bad_rows = df[df["has_bad_text"]]
print(f"Rows with weird characters: {len(bad_rows)} / {len(df)}")

Rows with weird characters: 3800 / 74141


In [136]:
total_bad_rows = df["has_bad_text"].sum()
print(f"Total rows with bad characters: {total_bad_rows}")
print(f"Percentage: {100 * total_bad_rows / len(df):.2f}%")

#preview a few examples
df[df["has_bad_text"]].head(3)[["author", "genre", "title"]]

Total rows with bad characters: 3800
Percentage: 5.13%


,author,genre,title
16,"Dan Sleigh,AndrÃ© Brink","Historical,Historical Fiction,Cultural,Africa,...",Islands
86,"Lotte Hammer,SÃ¸ren Hammer","Mystery,Crime,Mystery,Thriller,Fiction,Europea...",Alting har sin pris
97,Rolf MÃ¼ller,"Science,Mathematics,Childrens,Picture Books,Ch...",How Big Is a Foot?


In [137]:
df = df[~df["has_bad_text"]].reset_index(drop=True)

In [138]:
# flatten lists and normalize casing + whitespace
all_authors = sorted(
    set(
        a.strip().title() for sublist in df["author_list"] for a in sublist if a.strip()
    )
)
all_genres = sorted(
    set(g.strip().title() for sublist in df["genre_list"] for g in sublist if g.strip())
)

authors_df = pd.DataFrame({"Name": all_authors})
genres_df = pd.DataFrame({"Name": all_genres})

# assign primary keys
authors_df["AuthorID"] = range(1, len(authors_df) + 1)
genres_df["GenreID"] = range(1, len(genres_df) + 1)

print(f"Unique authors: {len(authors_df)}")
print(f"Unique genres: {len(genres_df)}")

Unique authors: 61317
Unique genres: 1153


In [139]:
authors_df.tail(10)

,Name,AuthorID
61307,Zsuzsi Gartner,61308
61308,Zulfikar Ghose,61309
61309,Zurab Karumidze,61310
61310,Zurab Lezhava,61311
61311,Zuri Day,61312
61312,Zvi Eckstein,61313
61313,Zvonko G. Vranesic,61314
61314,Zygmunt Bauman,61315
61315,Zygmunt Haupt,61316
61316,Zygmunt Miloszewski,61317


In [140]:
genres_df.tail()

,Name,GenreID
1148,Yuri,1149
1149,Zambia,1150
1150,Zen,1151
1151,Zimbabwe,1152
1152,Zombies,1153


In [141]:
authors = [a for sublist in df["author_list"] for a in sublist if a.strip()]
genres = [g for sublist in df["genre_list"] for g in sublist if g.strip()]

def is_weird(s):
    try:
        s.encode("ascii")
        return False  # clean
    except UnicodeEncodeError:
        return True  # has weird chars


weird_authors = [a for a in all_authors if is_weird(a)]
weird_genres = [g for g in all_genres if is_weird(g)]

print(f"Total authors: {len(authors)} | Weird authors: {len(weird_authors)}")
print(f"Total genres: {len(genres)} | Weird genres: {len(weird_genres)}")

Total authors: 102896 | Weird authors: 0
Total genres: 655513 | Weird genres: 0


In [142]:
books_df = df[
    [
        "title",
        "desc",
        "bookformat",
        "pages",
        "rating",
        "totalratings",
        "reviews",
        "isbn",
        "isbn13",
        "img",
        "link",
    ]
].copy()

# rename columns to match DB schema naming
books_df = books_df.rename(
    columns={
        "title": "Title",
        "desc": "Description",
        "bookformat": "BookFormat",
        "pages": "Pages",
        "rating": "AverageRating",
        "totalratings": "TotalRatings",
        "reviews": "ReviewsCount",
        "isbn": "ISBN",
        "isbn13": "ISBN13",
        "img": "ImageURL",
        "link": "GoodreadsLink",
    }
)

# assign primary key manually
books_df["BookID"] = range(1, len(books_df) + 1)

books_df.head(3)

,Title,Description,BookFormat,Pages,AverageRating,TotalRatings,ReviewsCount,ISBN,ISBN13,ImageURL,GoodreadsLink,BookID
0,Between Two Fires: American Indians in the Civ...,Reveals that several hundred thousand Indians ...,Hardcover,0,3.52,33,5,002914180X,9.78E+12,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/1001053.Betwee...,1
1,Fashion Sourcebook 1920s,Fashion Sourcebook - 1920s is the first book i...,Paperback,576,4.51,41,6,1906863482,9.78E+12,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/10010552-fashi...,2
2,Hungary 56,The seminal history and analysis of the Hungar...,Paperback,124,4.15,26,2,948984147,9.78E+12,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/1001077.Hungar...,3


In [143]:
# quick lookup dict: Author name → AuthorID
author_id_map = dict(zip(authors_df["Name"], authors_df["AuthorID"]))

book_author_rows = []

for idx, row in df.iterrows():
    book_id = idx + 1
    for author in row["author_list"]:
        name = author.strip().title()
        if name in author_id_map:
            book_author_rows.append(
                {"BookID": book_id, "AuthorID": author_id_map[name]}
            )

book_author_df = pd.DataFrame(book_author_rows)
book_author_df.head()

,BookID,AuthorID
0,1,34473
1,2,9025
2,2,16995
3,3,2969
4,4,7814


In [144]:
print(f"BookAuthor rows: {len(book_author_df)}")

BookAuthor rows: 102896


In [145]:
# quick lookup dict: Genre name → GenreID
genre_id_map = dict(zip(genres_df["Name"], genres_df["GenreID"]))

book_genre_rows = []

for idx, row in df.iterrows():
    book_id = idx + 1
    for genre in row["genre_list"]:
        name = genre.strip().title()
        if name in genre_id_map:
            book_genre_rows.append({"BookID": book_id, "GenreID": genre_id_map[name]})

book_genre_df = pd.DataFrame(book_genre_rows)
book_genre_df.head()

,BookID,GenreID
0,1,522
1,1,706
2,1,210
3,1,56
4,1,53


In [146]:
print(f"BookGenre rows: {len(book_genre_df)}")

BookGenre rows: 655513


In [147]:
# user table (for now just one admin)
users_df = pd.DataFrame(
    [
        {
            "UserID": 1,
            "Email": "admin@bookshelf.com",
            "PasswordHash": "hashed_password_here",
            "DisplayName": "Admin",
            "Role": "ADMIN",
            "CreatedAt": datetime.datetime.now(),
        }
    ]
)

# empty reading list
reading_list_df = pd.DataFrame(
    columns=[
        "UserID",
        "BookID",
        "Status",
        "ProgressPages",
        "UserRating",
        "Note",
        "AddedAt",
    ]
)

In [148]:
books_df.to_csv("../data/clean_data/books_table.csv", index=False)
authors_df.to_csv("../data/clean_data/authors_table.csv", index=False)
genres_df.to_csv("../data/clean_data/genres_table.csv", index=False)
book_author_df.to_csv("../data/clean_data/book_author_table.csv", index=False)
book_genre_df.to_csv("../data/clean_data/book_genre_table.csv", index=False)
users_df.to_csv("../data/clean_data/users_table.csv", index=False)
reading_list_df.to_csv("../data/clean_data/reading_list_table.csv", index=False)

In [149]:
print("Total unique books:", books_df["BookID"].nunique())

Total unique books: 70341


In [150]:
print("Unique ISBN values:", df["isbn"].nunique())

Unique ISBN values: 70341
